<a href="https://colab.research.google.com/github/GuilhermeOchoa/Data-Science/blob/master/Jogo_da_Velha_entre_IA's.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import random

class Sucessor:
    def __init__(self, estado, valor, coluna=None):
        self.estado = estado
        self.valor = valor
        self.coluna = coluna

    def get_estado(self):
        return self.estado

    def get_valor(self):
        return self.valor

    def get_coluna(self):
        return self.coluna

    def __str__(self):
        return f"Estado: {self.estado}, Valor: {self.valor}, Coluna: {self.coluna}"

class Minimax:
    def __init__(self, estado):
        self.estado = estado

    def get_melhor(self):
        melhor_sucessor = self.algoritmo(self.estado, False, self.livres(self.estado))
        return melhor_sucessor.coluna if melhor_sucessor else None  # Retorna o índice da melhor jogada

    def get_melhor_ab(self):
        melhor_sucessor = self.algoritmo_ab(self.estado, False, self.livres(self.estado), -999, 999)
        return melhor_sucessor.coluna if melhor_sucessor else None  # Retorna o índice da melhor jogada

    def livres(self, estado):
        return estado.count('0')

    def jogada_aleatoria(self):
        posicoes_livres = [i for i, v in enumerate(self.estado) if v == '0']
        return random.choice(posicoes_livres) if posicoes_livres else None

    def gera_vizinhos(self, estado, caracter):
        vizinhos = []
        for i in range(9):
            if estado[i] == '0':
                novo_estado = estado[:]
                novo_estado[i] = caracter
                vizinhos.append((novo_estado, i))  # Retorna o estado e o índice da jogada
        return vizinhos

    def utilidade(self, atual, profundidade):
        if self.vencedor(atual, 'X'):
            return -1
        if self.vencedor(atual, 'O'):
            return 1
        if profundidade == 0:
            return 0
        return 100

    def vencedor(self, atual, caracter):
        # Verifica linhas, colunas e diagonais para vitória
        for i in range(3):
            if all(atual[i*3 + j] == caracter for j in range(3)) or all(atual[i + j*3] == caracter for j in range(3)):
                return True
        return (atual[0] == caracter and atual[4] == caracter and atual[8] == caracter) or \
               (atual[2] == caracter and atual[4] == caracter and atual[6] == caracter)

    def algoritmo(self, estado, jogador, profundidade):
        valor = self.utilidade(estado, profundidade)
        if valor != 100:
            return Sucessor(estado, valor)

        vizinhos = self.gera_vizinhos(estado, 'X' if jogador else 'O')
        melhor_sucessor = None
        if jogador:
            menor = 999
            for vizinho, indice in vizinhos:
                atual = self.algoritmo(vizinho, False, profundidade - 1)
                if atual.get_valor() < menor:
                    menor = atual.get_valor()
                    melhor_sucessor = Sucessor(vizinho, menor, coluna=indice)  # Armazena o índice
            return melhor_sucessor
        else:
            maior = -999
            for vizinho, indice in vizinhos:
                atual = self.algoritmo(vizinho, True, profundidade - 1)
                if atual.get_valor() > maior:
                    maior = atual.get_valor()
                    melhor_sucessor = Sucessor(vizinho, maior, coluna=indice)  # Armazena o índice
            return melhor_sucessor

    def algoritmo_ab(self, estado, jogador, profundidade, alfa, beta):
        valor = self.utilidade(estado, profundidade)
        if valor != 100:
            return Sucessor(estado, valor)

        vizinhos = self.gera_vizinhos(estado, 'X' if jogador else 'O')
        melhor_sucessor = None
        if jogador:
            menor = 999
            for vizinho, indice in vizinhos:
                atual = self.algoritmo_ab(vizinho, False, profundidade - 1, alfa, beta)
                if atual.get_valor() < menor:
                    menor = atual.get_valor()
                    melhor_sucessor = Sucessor(vizinho, menor, coluna=indice)  # Armazena o índice
                if menor < alfa:
                    return melhor_sucessor
                beta = min(beta, menor)
            return melhor_sucessor
        else:
            maior = -999
            for vizinho, indice in vizinhos:
                atual = self.algoritmo_ab(vizinho, True, profundidade - 1, alfa, beta)
                if atual.get_valor() > maior:
                    maior = atual.get_valor()
                    melhor_sucessor = Sucessor(vizinho, maior, coluna=indice)  # Armazena o índice
                if maior > beta:
                    return melhor_sucessor
                alfa = max(alfa, maior)
            return melhor_sucessor


In [ ]:
import numpy as np

# Configurações da Rede Neural
INPUT_SIZE = 9         # Número de entradas
HIDDEN_SIZE = 5        # Número de neurônios na camada oculta (ajustável)
OUTPUT_SIZE = 9        # Número de saídas

# Configurações do Algoritmo Genético
POPULATION_SIZE = 50   # Tamanho da população
GENERATIONS = 100      # Número de gerações
MUTATION_RATE = 0.1    # Taxa de mutação

# Função de ativação
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Definição da Rede Neural
class MLP:
    def __init__(self, hidden_size):
        # Inicializa pesos aleatórios para a camada oculta e de saída
        self.hidden_weights = np.random.uniform(-1, 1, (hidden_size, INPUT_SIZE + 1))
        self.output_weights = np.random.uniform(-1, 1, (OUTPUT_SIZE, hidden_size + 1))

    def forward(self, inputs):
        # Adiciona o bias às entradas
        inputs = np.append(inputs, 1)

        # Calcula a saída da camada oculta
        hidden_inputs = np.dot(self.hidden_weights, inputs)
        hidden_outputs = sigmoid(hidden_inputs)

        # Adiciona o bias à saída da camada oculta
        hidden_outputs = np.append(hidden_outputs, 1)

        # Calcula a saída final
        final_inputs = np.dot(self.output_weights, hidden_outputs)
        final_outputs = sigmoid(final_inputs)

        return final_outputs

# Função de avaliação de fitness
def fitness_function(network, target_outputs, inputs):
    predictions = network.forward(inputs)
    return -np.mean((predictions - target_outputs) ** 2)  # Erro médio quadrado como função de custo

# Algoritmo Genético para otimizar pesos
def genetic_algorithm():
    # Inicializa a população de redes neurais
    population = [MLP(HIDDEN_SIZE) for _ in range(POPULATION_SIZE)]

    for generation in range(GENERATIONS):
        # Avalia o fitness de cada indivíduo
        fitness_scores = []
        for network in population:
            fitness = fitness_function(network, target_outputs, inputs)
            fitness_scores.append(fitness)

        # Seleção dos melhores indivíduos
        sorted_population = [net for _, net in sorted(zip(fitness_scores, population), key=lambda x: x[0], reverse=True)]
        population = sorted_population[:POPULATION_SIZE // 2]

        # Recombinação e Mutação
        new_population = []
        while len(new_population) < POPULATION_SIZE:
            parent1, parent2 = np.random.choice(population, 2)
            child1, child2 = crossover(parent1, parent2)
            mutate(child1, MUTATION_RATE)
            mutate(child2, MUTATION_RATE)
            new_population.append(child1)
            new_population.append(child2)

        population = new_population
        print(f"Geração {generation} - Melhor Fitness: {max(fitness_scores)}")

    # Retorna o melhor indivíduo da população final
    return sorted_population[0]

# Função de cruzamento (crossover)
def crossover(parent1, parent2):
    child1, child2 = MLP(HIDDEN_SIZE), MLP(HIDDEN_SIZE)
    child1.hidden_weights = np.where(np.random.rand(*parent1.hidden_weights.shape) > 0.5, parent1.hidden_weights, parent2.hidden_weights)
    child1.output_weights = np.where(np.random.rand(*parent1.output_weights.shape) > 0.5, parent1.output_weights, parent2.output_weights)
    child2.hidden_weights = np.where(np.random.rand(*parent2.hidden_weights.shape) > 0.5, parent2.hidden_weights, parent1.hidden_weights)
    child2.output_weights = np.where(np.random.rand(*parent2.output_weights.shape) > 0.5, parent2.output_weights, parent1.output_weights)
    return child1, child2

# Função de mutação
def mutate(network, mutation_rate):
    for weight_matrix in [network.hidden_weights, network.output_weights]:
        mutation_mask = np.random.rand(*weight_matrix.shape) < mutation_rate
        weight_matrix += mutation_mask * np.random.uniform(-0.5, 0.5, weight_matrix.shape)

# Dados de entrada e saída desejada (exemplo)
inputs = np.random.rand(INPUT_SIZE)        # Dados de entrada de exemplo
target_outputs = np.random.rand(OUTPUT_SIZE)  # Saída desejada de exemplo

# Executa o algoritmo genético
melhor_rede = genetic_algorithm()

# Teste com a melhor rede
saida_final = melhor_rede.forward(inputs)
print("Saída final da melhor rede: ", saida_final)


Geração 0 - Melhor Fitness: -0.06669492710187726
Geração 1 - Melhor Fitness: -0.03427013864337049
Geração 2 - Melhor Fitness: -0.027323415718149745
Geração 3 - Melhor Fitness: -0.025896548741004447
Geração 4 - Melhor Fitness: -0.022339126455759827
Geração 5 - Melhor Fitness: -0.00765382236660307
Geração 6 - Melhor Fitness: -0.006728101290185456
Geração 7 - Melhor Fitness: -0.007497435909664683
Geração 8 - Melhor Fitness: -0.004502127669223306
Geração 9 - Melhor Fitness: -0.0031415849032537693
Geração 10 - Melhor Fitness: -0.002032702062325107
Geração 11 - Melhor Fitness: -0.0015502161146552283
Geração 12 - Melhor Fitness: -0.002073901056250159
Geração 13 - Melhor Fitness: -0.0027683381675471853
Geração 14 - Melhor Fitness: -0.002216148046733979
Geração 15 - Melhor Fitness: -0.0016431140207490086
Geração 16 - Melhor Fitness: -0.0010779281366100888
Geração 17 - Melhor Fitness: -0.0025408879471074813
Geração 18 - Melhor Fitness: -0.0012438469611714298
Geração 19 - Melhor Fitness: -0.00083

In [4]:

import random
def inicializar_tabuleiro():
    return ['0' for _ in range(9)]

def imprimir_tabuleiro(tabuleiro):
    print(f"{tabuleiro[0]} | {tabuleiro[1]} | {tabuleiro[2]}")
    print("-" * 10)
    print(f"{tabuleiro[3]} | {tabuleiro[4]} | {tabuleiro[5]}")
    print("-" * 10)
    print(f"{tabuleiro[6]} | {tabuleiro[7]} | {tabuleiro[8]}")

def verificar_estado(tabuleiro):
    jogo = Minimax(tabuleiro)
    if jogo.vencedor(tabuleiro, 'X'):
        return 1
    elif jogo.vencedor(tabuleiro, 'O'):
        return 2
    elif '0' not in tabuleiro:
        return 3
    return 0

def jogada_ia(tabuleiro, dificuldade):
    jogo_ia = Minimax(tabuleiro)

    if dificuldade == 1:  # Nível 1: 25% Minimax, 75% Aleatório
        probabilidade_minimax = 0.25
    elif dificuldade == 2:  # Nível 2: 50% Minimax, 50% Aleatório
        probabilidade_minimax = 0.5
    else:  # Nível 3: 100% Minimax
        probabilidade_minimax = 1.0

    if random.random() < probabilidade_minimax:
        movimento_ia = jogo_ia.get_melhor()  # Pega o índice da jogada
    else:
        movimento_ia = jogo_ia.jogada_aleatoria()

    tabuleiro[movimento_ia] = 'O'
    print(f"IA escolheu a posição {movimento_ia}")

def jogada_iaMLP(tabuleiro):
    jogo_iaMLP = MLP(tabuleiro)
    movimento_iaMLP = jogo_iaMLP.get_melhor()
    tabuleiro[movimento_iaMLP] = 'X'
    print(f"IA MLP escolheu a posição {movimento_iaMLP}")



def jogar():
    num_partidas = int(input("Quantas partidas você deseja jogar? "))
    dificuldade = int(input("Escolha a dificuldade (1: Fácil, 2: Médio, 3: Difícil): "))

    # Contadores para vitórias e empates
    vitorias_jogador = 0
    vitorias_ia = 0
    empates = 0

    for partida in range(num_partidas):
        print(f"\n--- Partida {partida + 1} ---")
        tabuleiro = inicializar_tabuleiro()
        jogador_humano = 'X'

        while True:
            imprimir_tabuleiro(tabuleiro)
            movimento = int(input("Escolha sua posição (0-8): "))
            if tabuleiro[movimento] == '0':
                tabuleiro[movimento] = jogador_humano
            else:
                print("Posição inválida! Tente novamente.")
                continue

            estado = verificar_estado(tabuleiro)
            if estado == 1:
                imprimir_tabuleiro(tabuleiro)
                print("Você venceu!")
                vitorias_jogador += 1
                break
            elif estado == 2:
                imprimir_tabuleiro(tabuleiro)
                print("A IA venceu!")
                vitorias_ia += 1
                break
            elif estado == 3:
                imprimir_tabuleiro(tabuleiro)
                print("Empate!")
                empates += 1
                break

            jogada_ia(tabuleiro, dificuldade)

            estado = verificar_estado(tabuleiro)
            if estado == 1:
                imprimir_tabuleiro(tabuleiro)
                print("Você venceu!")
                vitorias_jogador += 1
                break
            elif estado == 2:
                imprimir_tabuleiro(tabuleiro)
                print("A IA venceu!")
                vitorias_ia += 1
                break
            elif estado == 3:
                imprimir_tabuleiro(tabuleiro)
                print("Empate!")
                empates += 1
                break

        imprimir_tabuleiro(tabuleiro)

    # Exibe o resumo das partidas
    print("\n--- Resultado Final ---")
    print(f"Vitórias do jogador: {vitorias_jogador}")
    print(f"Vitórias da IA: {vitorias_ia}")
    print(f"Empates: {empates}")
# Executa o jogo
jogar()


Quantas partidas você deseja jogar? 1
Escolha a dificuldade (1: Fácil, 2: Médio, 3: Difícil): 3

--- Partida 1 ---
0 | 0 | 0
----------
0 | 0 | 0
----------
0 | 0 | 0
Escolha sua posição (0-8): 0
IA escolheu a posição 4
X | 0 | 0
----------
0 | O | 0
----------
0 | 0 | 0
Escolha sua posição (0-8): 3
IA escolheu a posição 6
X | 0 | 0
----------
X | O | 0
----------
O | 0 | 0
Escolha sua posição (0-8): 2
IA escolheu a posição 1
X | O | X
----------
X | O | 0
----------
O | 0 | 0
Escolha sua posição (0-8): 7
IA escolheu a posição 5
X | O | X
----------
X | O | O
----------
O | X | 0
Escolha sua posição (0-8): 8
X | O | X
----------
X | O | O
----------
O | X | X
Empate!
X | O | X
----------
X | O | O
----------
O | X | X

--- Resultado Final ---
Vitórias do jogador: 0
Vitórias da IA: 0
Empates: 1
